In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('data/cleaned_rounds_data.csv')
df.head()

,round_winner,player_1_team_name,player_1_name,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,1,G2 Esports,NiKo,0,1.0,1.0,0,0,0.0,700.0,...,0,0,0,0,1,0,1,1,0,0
1,1,G2 Esports,NiKo,0,1.0,2.0,0,1,100.0,4100.0,...,0,0,0,0,1,1,1,1,0,0
2,0,G2 Esports,NiKo,0,1.0,3.0,0,1,100.0,5100.0,...,0,0,0,0,1,1,0,1,0,0
3,1,G2 Esports,NiKo,0,3.0,4.0,0,1,100.0,2800.0,...,0,0,0,0,1,1,0,1,0,0
4,1,G2 Esports,NiKo,0,4.0,5.0,0,0,0.0,200.0,...,0,0,0,0,1,1,1,1,0,0


In [3]:
df = df.drop(columns=[f'player_{i}_{suffix}' for i in range(1, 11) for suffix in ['team_name', 'name']])
df.head()

,round_winner,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,player_1_balance,player_1_AK-47,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,1,0,1.0,1.0,0,0,0.0,700.0,1650.0,0,...,0,0,0,0,1,0,1,1,0,0
1,1,0,1.0,2.0,0,1,100.0,4100.0,150.0,0,...,0,0,0,0,1,1,1,1,0,0
2,0,0,1.0,3.0,0,1,100.0,5100.0,1000.0,1,...,0,0,0,0,1,1,0,1,0,0
3,1,0,3.0,4.0,0,1,100.0,2800.0,400.0,0,...,0,0,0,0,1,1,0,1,0,0
4,1,0,4.0,5.0,0,0,0.0,200.0,3100.0,0,...,0,0,0,0,1,1,1,1,0,0


In [4]:
X = df.drop(['round_winner'], axis=1)
X.head()

,player_1_team_num,player_1_kills_total,player_1_deaths_total,player_1_has_defuser,player_1_has_helmet,player_1_armor_value,player_1_current_equip_value,player_1_balance,player_1_AK-47,player_1_M4A1-S,...,player_10_CZ75-Auto,player_10_Desert Eagle,player_10_R8 Revolver,player_10_Dual Berettas,player_10_Smoke Grenade,player_10_High Explosive Grenade,player_10_Flashbang,player_10_Incendiary Grenade,player_10_Molotov,player_10_Decoy Grenade
0,0,1.0,1.0,0,0,0.0,700.0,1650.0,0,0,...,0,0,0,0,1,0,1,1,0,0
1,0,1.0,2.0,0,1,100.0,4100.0,150.0,0,0,...,0,0,0,0,1,1,1,1,0,0
2,0,1.0,3.0,0,1,100.0,5100.0,1000.0,1,0,...,0,0,0,0,1,1,0,1,0,0
3,0,3.0,4.0,0,1,100.0,2800.0,400.0,0,0,...,0,0,0,0,1,1,0,1,0,0
4,0,4.0,5.0,0,0,0.0,200.0,3100.0,0,0,...,0,0,0,0,1,1,1,1,0,0


In [5]:
y = df['round_winner']

In [6]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [14]:
best_params = {
    'max_depth': 9,
    'learning_rate': 0.05,
    'gamma': 0.0,
    'reg_lambda': 0.1,
    'scale_pos_weight': 1.0
}

PERFORMANCE OF THE MODEL USING THE OPTIMAL PARAMETERS

In [15]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=10000, **best_params)
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print("Average ROC-AUC: ", scores.mean())

Average ROC-AUC:  0.8746187373799262


In [16]:
X_columns = df.drop(['round_winner'], axis=1).columns

In [17]:
model.fit(X_train, y_train)
importances = pd.Series(model.feature_importances_, index=X_columns)
importances = importances.sort_values(ascending=False)
print(importances)

player_2_Incendiary Grenade     0.026443
player_6_has_defuser            0.024662
player_5_has_defuser            0.021038
player_2_has_helmet             0.015989
player_10_Incendiary Grenade    0.014476
                                  ...   
player_8_G3SG1                  0.000000
player_8_MP5-SD                 0.000000
player_8_UMP-45                 0.000000
player_8_P90                    0.000000
player_10_Decoy Grenade         0.000000
Length: 470, dtype: float32


In [18]:
y_pred = model.predict(X_test)

In [19]:
print(classification_report(y_test, y_pred))
confusion_m = confusion_matrix(y_test, y_pred)
conf_matrix_df = pd.DataFrame(confusion_m, index=['Actual Negative', 'Actual Positive'],
                              columns=['Predicted Negative', 'Predicted Positive'])
print(conf_matrix_df)

print({
    'true negatives': f'{round(100*confusion_m[0, 0]/len(y_test))}%',
    'false positives': f'{round(100*confusion_m[0, 1]/len(y_test))}%',
    'false negatives': f'{round(100*confusion_m[1, 0]/len(y_test))}%',
    'true positives': f'{round(100*confusion_m[1, 1]/len(y_test))}%'
})

              precision    recall  f1-score   support

           0       0.80      0.81      0.81      2270
           1       0.82      0.81      0.81      2405

    accuracy                           0.81      4675
   macro avg       0.81      0.81      0.81      4675
weighted avg       0.81      0.81      0.81      4675

                 Predicted Negative  Predicted Positive
Actual Negative                1837                 433
Actual Positive                 455                1950
{'true negatives': '39%', 'false positives': '9%', 'false negatives': '10%', 'true positives': '42%'}


PERFORMANCE OF THE MODEL WITHOUT THE OPTIMAL PARAMETERS FOUND

In [8]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', learning_rate=0.1, n_estimators=10000)
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc')
print("Average ROC-AUC: ", scores.mean())

Average ROC-AUC:  0.8698486257367379


In [9]:
X_columns = df.drop(['round_winner'], axis=1).columns

In [10]:
model.fit(X_train, y_train)
importances = pd.Series(model.feature_importances_, index=X_columns)
importances = importances.sort_values(ascending=False)
print(importances)

player_2_Incendiary Grenade    0.021082
player_5_has_defuser           0.018076
player_6_has_defuser           0.015055
player_3_has_defuser           0.014264
player_9_FAMAS                 0.013211
                                 ...   
player_8_Scar-20               0.000000
player_8_G3SG1                 0.000000
player_8_MP5-SD                0.000000
player_8_UMP-45                0.000000
player_2_R8 Revolver           0.000000
Length: 470, dtype: float32


In [11]:
y_pred = model.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))
confusion_m = confusion_matrix(y_test, y_pred)
conf_matrix_df = pd.DataFrame(confusion_m, index=['Actual Negative', 'Actual Positive'],
                              columns=['Predicted Negative', 'Predicted Positive'])
print(conf_matrix_df)

print({
    'true negatives': f'{round(100*confusion_m[0, 0]/len(y_test))}%',
    'false positives': f'{round(100*confusion_m[0, 1]/len(y_test))}%',
    'false negatives': f'{round(100*confusion_m[1, 0]/len(y_test))}%',
    'true positives': f'{round(100*confusion_m[1, 1]/len(y_test))}%'
})

              precision    recall  f1-score   support

           0       0.79      0.81      0.80      2270
           1       0.82      0.80      0.81      2405

    accuracy                           0.80      4675
   macro avg       0.80      0.80      0.80      4675
weighted avg       0.80      0.80      0.80      4675

                 Predicted Negative  Predicted Positive
Actual Negative                1837                 433
Actual Positive                 484                1921
{'true negatives': '39%', 'false positives': '9%', 'false negatives': '10%', 'true positives': '41%'}
